In [4]:
# @ copyright
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import dask.dataframe as dd

import gc
warnings.simplefilter('ignore')
matplotlib.rcParams['figure.dpi'] = 100
sns.set()
%matplotlib inline


### to save data or model
import pickle
import h5py


     The aim of this notebook is to provide investogation of the data sets
     as a first requirement for building a model. Note that there was computationally limited resources (using a    PC).So, I always think of the simplest and cost effective way to go about achieving a task 

### Loading the dataset


[If you have the merged data set file then click here to skip this part](#Merged-Data)


<p> Note that the test data have been committed out but similar steps would be replicated for the test data. </p>

 The goal here is to prepare the training data set for
 training.

In [4]:
# read the training set
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')


In [4]:
# read the building data set
building = pd.read_csv('building_metadata.csv')


 Merging Train and building data sets that have been imported from above.
 
 

In [5]:
train = train.merge(building, on='building_id', how='left')
#test = test.merge(building, on='building_id', how='left')
#test.to_pickle('test_building')  # where to save it, usually as a .pkl

In [11]:
# Saving the data set as pickle file to enable easy loading of merged data set
# This also saves the cost of merging
train.to_pickle('train_building')  # where to save it, usually as a .pkl


# Merged Data


In [5]:
# load merged building and train data
mergerd = pd.read_pickle('train_building')

In [6]:
# load the weather data
weather_train = pd.read_csv('weather_train.csv')
#weather_test = pd.read_csv('weather_test.csv')

In [7]:
# merged to produce train-building-weather data set
train = mergerd.merge(weather_train, on=['site_id', 'timestamp'], how='left')


In [5]:
# check the number of columns present in data set
train.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'year_built', 'floor_count',
       'air_temperature', 'cloud_coverage', 'dew_temperature',
       'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction',
       'wind_speed'],
      dtype='object')

Let us investigate if it is possible to parition data based on meter, site_id and primary_use 

    1) We have to check the  unique elements present in each of the 3 columns named above

In [6]:
# unique elements  in the column 'primary_use
train.primary_use.unique() 


array(['Education', 'Lodging/residential', 'Office',
       'Entertainment/public assembly', 'Other', 'Retail', 'Parking',
       'Public services', 'Warehouse/storage', 'Food sales and service',
       'Religious worship', 'Healthcare', 'Utility', 'Technology/science',
       'Manufacturing/industrial', 'Services'], dtype=object)

In [13]:
# unique element in the column meter 
train.meter.unique()

array([0, 3, 1, 2])

In [15]:
# unique element in the column site id
train.site_id.unique() # the numbers of site id present

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

<p> we observe that this will create a lot of smaller subset and it might not be possible to make a model for each
    of this (for computational limitation and flexibility reasons) </p>
    
Nevertheless we will proceed to see how the partitioning could be done

# Partitioning
    
    1) Primary_use column is an pandas object type (or say it is not a numeric value).
    2) Convert it to pandas category type and  assign a numeric value to each category. The category mappings              can be trivially retrieved in pandas
    3) Meter and site_id are both int64 type. Therefore nopthing to be done here
    4) It is possible to then use Multi index approach to index the data set on primery_use,
       meter and site_id
    5) However, let us use a simple approach. A new column will be added to the data set and it
       will be called separator. To achieve this convert meter, site_id, and primary_use to string data type.
    6) The column separator is defined as:
          separator  = meter + primary_use  + site_id
    7) Now we can select a subset or partition based on this selector. Data points belonging to the same
       separator value would share same meter id, primary use and site id.

In [8]:
## step 1 : convert pry_use to category set to 
train["primary_use"] = train["primary_use"].astype('category')

In [8]:
# check that primary use is of type category
# this step does not change the label for the contents of pimary  use
train.dtypes # primary use is now a category

building_id              int64
meter                    int64
timestamp               object
meter_reading          float64
site_id                  int64
primary_use           category
square_feet              int64
year_built             float64
floor_count            float64
air_temperature        float64
cloud_coverage         float64
dew_temperature        float64
precip_depth_1_hr      float64
sea_level_pressure     float64
wind_direction         float64
wind_speed             float64
dtype: object

In [9]:
# encode each category as numeric values
train["primary_use_encodings"] = train["primary_use"].cat.codes


In [10]:
# cheeck the encodings oe mappings that have been assigned
train.primary_use_encodings.unique() # the encodings

array([ 0,  4,  6,  1,  7, 11,  8,  9, 15,  2, 10,  3, 14, 13,  5, 12])

In [11]:
# Now step 5
# step 5 : convert meter from int64 to string
train['meter']= train['meter'].values.astype(np.str)

In [9]:
# convert site_id to string too
train['site_id']= train['site_id'].values.astype(np.str) 

In [12]:
# convert pry use  to string
train['primary_use_encodings']= train['primary_use_encodings'].values.astype(np.str)

In [17]:
# step 6 : separator column defined here 
train['separator'] = train['meter'] + train['site_id']+ train['primary_use_encodings']

In [24]:
unique_partition = train.separator.unique()

In [33]:
# these are the unique partions or groups available
# it is very diffcult to create a model for each of these
# read this as 000: meter 0 , site 0, primary use 0 (education)
# 004 : meter 0 , site id 0, pry use 4 (Lodging/residential)
# to get the mapping of pry use check next cell
unique_partition

array(['000', '004', '006', '001', '007', '0011', '008', '010', '310',
       '014', '011', '019', '319', '016', '316', '029', '026', '020',
       '021', '120', '320', '0215', '024', '124', '324', '0211', '1211',
       '126', '326', '121', '321', '022', '122', '322', '0210', '129',
       '028', '329', '023', '123', '0214', '1214', '039', '036', '031',
       '030', '034', '033', '0311', '0315', '038', '037', '0310', '040',
       '0413', '044', '049', '041', '0414', '048', '057', '050', '051',
       '053', '054', '055', '059', '056', '060', '260', '066', '160',
       '064', '266', '264', '061', '161', '261', '164', '069', '170',
       '270', '070', '370', '086', '081', '087', '089', '0815', '096',
       '196', '094', '194', '294', '090', '290', '190', '091', '191',
       '291', '0912', '099', '199', '299', '296', '0100', '1100', '3100',
       '0107', '1107', '0101', '0106', '01013', '11013', '31013', '3101',
       '0104', '0110', '3110', '1110', '0120', '01211', '0126', '0121

#### Mappings for primary use

In [37]:
 # store the encodings/numbers  of the pry use
labels = train.primary_use.unique() #munique elements of the sites (how many sites we have)


In [41]:
# for each number retrieve its category and store as tuple in a list

category_mappings=[] # list for storing number and cateogory pair
for i in labels:
    temp = train.primary_use.cat.categories.get_loc(i) # retrieve the category given the number
    category_mappings.append((temp,i)) # store the mappings
    

In [42]:
# check all the mappings categories
category_mappings 

[(0, 'Education'),
 (4, 'Lodging/residential'),
 (6, 'Office'),
 (1, 'Entertainment/public assembly'),
 (7, 'Other'),
 (11, 'Retail'),
 (8, 'Parking'),
 (9, 'Public services'),
 (15, 'Warehouse/storage'),
 (2, 'Food sales and service'),
 (10, 'Religious worship'),
 (3, 'Healthcare'),
 (14, 'Utility'),
 (13, 'Technology/science'),
 (5, 'Manufacturing/industrial'),
 (12, 'Services')]

In [46]:
# save data as csv
train.to_csv('AllData.csv')

In [4]:
#train = pd.read_csv('AllData.csv')

# Further Details

If we have 0112, it might be difficult to know whether the site id is 1 and category is 12 or vice versa.
So, I have decided to convert site id to alphabets to remove this ambiguity.
The result of this is same as when site id is not chneged but this is easier to read.

In [13]:
# Site_id as Alphabets
train["site_id"] = train["site_id"].astype('category')


In [52]:
#siteIdUnique = train["site_id"].unique()

In [14]:
 # this is what site id will be mapped to
newSiteName = ['a','b','c','d','e','f','g','h','i','j', 'k', 'l','m','n','o','p']
# change site id to category.and assign the mapping from the list aboce
train.site_id.cat.rename_categories(newSiteName,inplace=True)

In [15]:
# site id was chneged to str for concatenation
train['site_id']= train['site_id'].values.astype(np.str) # convert siteid back to string

In [59]:
# delete previous separator
del train['separator']

In [16]:
# make new separator column
train['separator'] = train['meter'] + train['site_id']+ train['primary_use_encodings']

In [18]:
# check the unique separator values which corresponds to the partiton
separator = train.separator.unique()

In [21]:
# check the partition
separator

array(['0a0', '0a4', '0a6', '0a1', '0a7', '0a11', '0a8', '0b0', '3b0',
       '0b4', '0b1', '0b9', '3b9', '0b6', '3b6', '0c9', '0c6', '0c0',
       '0c1', '1c0', '3c0', '0c15', '0c4', '1c4', '3c4', '0c11', '1c11',
       '1c6', '3c6', '1c1', '3c1', '0c2', '1c2', '3c2', '0c10', '1c9',
       '0c8', '3c9', '0c3', '1c3', '0c14', '1c14', '0d9', '0d6', '0d1',
       '0d0', '0d4', '0d3', '0d11', '0d15', '0d8', '0d7', '0d10', '0e0',
       '0e13', '0e4', '0e9', '0e1', '0e14', '0e8', '0f7', '0f0', '0f1',
       '0f3', '0f4', '0f5', '0f9', '0f6', '0g0', '2g0', '0g6', '1g0',
       '0g4', '2g6', '2g4', '0g1', '1g1', '2g1', '1g4', '0g9', '1h0',
       '2h0', '0h0', '3h0', '0i6', '0i1', '0i7', '0i9', '0i15', '0j6',
       '1j6', '0j4', '1j4', '2j4', '0j0', '2j0', '1j0', '0j1', '1j1',
       '2j1', '0j12', '0j9', '1j9', '2j9', '2j6', '0k0', '1k0', '3k0',
       '0k7', '1k7', '0k1', '0k6', '0k13', '1k13', '3k13', '3k1', '0k4',
       '0l0', '3l0', '1l0', '0m0', '0m11', '0m6', '0m13', '0m9', '0m1',
 

In [23]:
len(separator) # total groups

221

In [38]:
# to retieve a particular partition use the syntax below
# I ran out of memory here so I cant run to show sample results
# train.query('separator == "0f1"')[1:2]
# train.loc[train['separator'] == '0f1'][1:2]

### Conclusion

This isuse if there are enough computational resources. It might be possible to make plots and compare partions
But in the next step the data set will be divided based on meter in order to have a smaller subset and build models 
with those.